In [7]:
import pandas as pd

In [8]:
fold1_X_train = pd.read_csv('data/processed/fold_1_X_train.csv')
fold1_X_test = pd.read_csv('data/processed/fold_1_X_test.csv')  
fold1_y_train = pd.read_csv('data/processed/fold_1_y_train.csv')
fold1_y_test = pd.read_csv('data/processed/fold_1_y_test.csv')
fold2_X_train = pd.read_csv('data/processed/fold_2_X_train.csv')
fold2_X_test = pd.read_csv('data/processed/fold_2_X_test.csv')  
fold2_y_train = pd.read_csv('data/processed/fold_2_y_train.csv')
fold2_y_test = pd.read_csv('data/processed/fold_2_y_test.csv')

print("Ready for Feature Classification:")
print(f"  Training features: {fold1_X_train.shape}")
print(f"  Features to classify: {len(fold1_X_train.columns)}")
print(f"  Data integrity verified: ✅")

Ready for Feature Classification:
  Training features: (8495, 18)
  Features to classify: 18
  Data integrity verified: ✅
